In [1]:
import numpy as np
import pandas as pd
import os
import multiprocessing as mul
from multiprocessing import Process

## IMPORTING AND SORTING THE DATA

WE HAVE CHOSEN ICECUBE 2008-18 DATA FOR THIS STUDY


THE ICE CUBE DATA SET HAS 1134450 NEUTRINO EVENTS


We select neutrino events with Energy >= 100TeV = 10^5 GeV

i.e log10(E/GeV) > 5


There are 192107 such neutrino events in this data

The ms pulsars are taken from the ATNF Catalogue

There are 441 pulsars (as of May 2022 when the study started)

All the pulsars lie in the declination range of -87 to +87 degrees

In [2]:
####
#### IMPORTING AND SPLITTING ICDATA $$$


path = "/media/darkwake/VIB2/Project-IceCube/icecube_10year_ps/events"
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv",
"IC86_III_exp.csv", "IC86_IV_exp.csv", "IC86_V_exp.csv", "IC86_VI_exp.csv", "IC86_VII_exp.csv"]
file = filenames[0]
f = open(os.path.join(path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
content = []
for file in filenames:
    f = open(os.path.join(path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()
icdata = pd.DataFrame(content, columns=column)
icdata['log10(E/GeV)'] = [float(i) for i in icdata['log10(E/GeV)']]

icdata = icdata.sort_values('log10(E/GeV)')
icdata = icdata.reset_index()
icdata = icdata.drop('index', axis=1)
icdata2 = icdata[icdata['log10(E/GeV)'] > 5]
icdata2 = icdata2.reset_index()
icdata2 = icdata2.drop('index', axis=1)
icdata2

#IMPORTING MSPDATA
f = open("/media/darkwake/VIB2/Project-IceCube/10milsecpsr.txt", 'r')
lines = f.readlines()

content=[]
column=lines[3].split()
for line in lines[:]:
    content.append(line.split())
    #the INITAL DATABASE IS CLUTTERED SO WE REMOVE THE NULL COLUMNS AND OTHER CLUTTER
mspdata = pd.DataFrame(content).drop(range(0,6)).dropna().drop([2,6,8,10,11,13,14], axis=1)
f.close()
line = []
lines = []

mspdata.columns = column
column = []
content=[]
#mspdata = mspdata.sort_values('DECJD')
#mspdata.dropna(inplace=True)
#mspdata = mspdata.reset_index()
#mspdata = mspdata.drop(["index", "#"], axis=1)

In [3]:
icdata2

,MJD[days],log10(E/GeV),AngErr[deg],RA[deg],Dec[deg],Azimuth[deg],Zenith[deg]
0,58192.76586876,5.01,0.20,304.076,-27.773,234.673,62.284
1,57692.17772634,5.01,0.30,328.288,-24.485,225.360,65.593
2,56499.25268749,5.01,0.20,215.304,-72.162,269.459,17.774
3,55742.58579947,5.01,0.29,121.980,-37.347,97.098,52.613
4,57168.08498961,5.01,0.65,209.099,-81.386,154.309,8.532
...,...,...,...,...,...,...,...
192102,57850.41489986,7.00,1.48,299.091,-19.242,135.893,70.801
192103,56616.28975047,7.01,1.24,300.952,-48.575,312.569,41.469
192104,57741.31320782,7.04,3.27,226.680,-7.559,64.185,82.379
192105,58099.46879008,7.26,3.32,317.013,-34.929,22.825,55.146


In [4]:
mspdata

,#,NAME,Gl,Gb,RAJD,DECJD,P0,F0,DIST
6,1,J0023+0923,111.383,-52.849,5.82032291,9.38996121,0.003050203104754390,327.8470205611185,1.818
7,2,J0024-7204aa,305.895,-44.889,6.02363,-72.08128,0.00184,543.478261,2.688
8,3,J0024-7204ab,305.891,-44.891,6.034006,-72.0798894,0.0037046394947985,269.93179806134,2.540
9,4,J0024-7204ac,305.898,-44.902,6.02363,-72.06806,0.00274,364.963504,4.690
10,5,J0024-7204ad,305.898,-44.902,6.02363,-72.06806,0.00374,267.379679,4.690
...,...,...,...,...,...,...,...,...,...
560,463,J2310-0555,69.702,-57.907,347.526792,-5.92664,0.0026125867919415,382.76240356282,1.556
561,464,J2317+1439,91.361,-42.360,349.288484923,14.65868362,0.0034452510723611,290.25460815391,1.667
562,465,J2322+2057,96.515,-37.310,350.5930632,20.9507437,0.0048084282894641,207.96816335831,1.011
564,466,J2322-2650,28.637,-70.228,350.64433350,-26.84953253,0.00346309917908790,288.758695112907,0.227


In [5]:
msra = [float(i) for i in mspdata['RAJD']]
msdec = [float(i) for i in mspdata['DECJD']]
extensions = 441 - len(icdata2)%441
icra = [float(i) for i in icdata2['RA[deg]']]
icdec = [float(i) for i in icdata2['Dec[deg]']]
icang = [float(i) for i in icdata2['AngErr[deg]']]
icra.extend([0]*extensions)
icdec.extend([0]*extensions)


Selecting events which are at an angular distance < $20^\circ$ from a particular ms psr source = $N$

In [6]:
def hvovec(lon1, lat1, lon2, lat2):

    #Convert decimal degrees to Radians:
    lon1 = np.deg2rad(lon1)
    lat1 = np.deg2rad(lat1)
    lon2 = np.deg2rad(lon2)
    lat2 = np.deg2rad(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)

    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    return np.abs(np.rad2deg(np.arccos(a)))

In [134]:
#all_angles = load.t2a2b(icra,icdec,msra,msdec,extensions,which='a')
lg = int(len(icra)/len(msra))
p = len(msra)  
#for k in range(lg):
k = list(range(0,lg))
def mul_angle_finder(k):
    st2a = []
    ilo = icra[k * p  :p * k + p]
    ila = icdec[k * p  :p * k + p]
    lo =[]
    la = []
    for j in range(p):#441
        lo = [msra[(i + j)%p] for i in range(0,p)]
        la = [msdec[(i + j)%p] for i in range(0,p)]
        st2a.append(hvovec(lo, la, ilo, ila))
        if k == lg - 1:
            for l in range(extensions, len(st2a)):
                st2a[-1][l] = -1
    return st2a
pool = mul.Pool()
op_async = pool.map_async(mul_angle_finder, range(0,lg))
all_angles_o = op_async.get()

In [136]:
all_angles_o[435]

[array([112.48843031,  75.37440557,  44.17800266,  35.47796299,
         69.34099571,  62.54625409,  54.93479894,  53.61498492,
         12.01448126,  10.60437427,  69.73312923,  22.74133315,
         42.77682877,  52.19011081,  22.29673603,  49.65544584,
         37.36483123,  28.61287887,  22.52124008,  50.55476852,
         37.60963639,  22.91313602,  34.21830592,  57.38216819,
         37.35539242,  40.79417723,  59.19838279,  52.96281552,
         86.83179213,  54.66469472,  76.12855767, 124.76829205,
        125.15691777, 113.45668093,  96.37505811, 136.4648952 ,
         75.64501563, 107.48575974,  46.10213192, 106.46313986,
         77.33063199, 144.74783912, 104.15433655,  63.84067258,
        125.14023547,  79.45187573,  71.43712343, 103.92997273,
         63.63296612, 135.83989186, 109.79422501,  53.937868  ,
         47.35792271, 105.23323164, 102.72748815, 109.13936627,
         95.53111814,  33.07701881, 151.30268753,  88.89441605,
        106.95454703,  53.62033667, 159.

In [166]:
angl = []
for i in range(len(msra)):
    temp = []
    for k in range(len(all_angles_o)):
        t2 = []
        for j in range(len(all_angles_o[k])):
            if all_angles_o[k][j][((+i-j) % 441)] != -1:
                t2.append(all_angles_o[k][j][((+i-j) % 441)])
            #temp.append(all_angles_o[k][j][((441+i-j) % 441)])
        t2 = [t2[(i+l)%len(t2)] for l in range(0,len(t2))]
        temp.extend(t2)
    angl.append(temp)



In [167]:
i=34
print([msra[i], msdec[i]])

[34.526494038, 42.53815976]


In [168]:
j=0
aaaa = hvovec(icra[j],icdec[j], msra[i], msdec[i])
aaaa

108.67255687808401

In [169]:
angl[i].index(aaaa)

0

In [86]:
extensions

169

In [97]:
for i in angl:
    a = i.count(None)
    if a != 0:
        print(a)

In [84]:
len(angl[i])

192276

### all_angles_o[k][j][$(441+i-j-1) \% 441$] = space angle between $p_i$  and $\nu_{kp+j}$


Selecting events which are at an angular distance < $20^\circ$ from a particular ms psr source = $N$

### $S_{ij}$ = $f( \nu$ event $i$ and  pulsar source $j)$

In [ ]:
def S_ij(i,j):
    ang = hvovec(icra[i], icdec[i], msra[j], msdec[j])
    if ang < 20:
        sg = icang[i] ** 2
        return np.exp(-1 * ang / (2 * sg)) / (2 * np.pi * sg)
    else:
        return -1

In [ ]:
S_ij(1,1)

In [ ]:
S = []
#for j in range(p):
def multi_si(j):
    aa = []
    for i in range(len(icdata2)):
        sij = S_ij(i, j)
        #if sij != -1:
        aa.append(sij)
    return aa

pool = mul.Pool()
op_async = pool.map_async(multi_si, range(0,p))
S = op_async.get()

In [ ]:
for i in S:
    print(max(S))

$B_i$ = 103.12813

For each sample, given the Dec δi of an event:

The background PDF is determined by the relative number of
events in δi ± 3◦ divided by the solid angle.


so calculate total no of events within delta +/- 3
and then divide by 2 pi (sin[delta+3]-sin[delta-3])

Solid angle span = $2\pi\cos(\sin(\delta + 6) - sin(\delta - 6))$

We estimate the background PDF by calculating the ratio of average number of events per steradian at each
declination and the total number of events. 

In [ ]:
def b_ivec(lon1, lat1, lon2, lat2):

    #Convert decimal degrees to Radians:
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)


    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    return (np.rad2deg(np.arccos(a)), a)

In [ ]:
evts_p_srad = []
l_ic = len(icdata2)
for i in range(len(mspdata)):
#returns $B_i$ as defined above
#def B_i(i):

    r = []
    psra = float(mspdata['RAJD'][i])
    psdec = float(mspdata['DECJD'][i])
    pssa = np.cos(np.cos(np.deg2rad(psdec) + np.cos(np.deg2rad(3))) - np.cos(np.deg2rad(psdec) - np.cos(np.deg2rad(3))))
    for j in range(l_ic):
        if icdec[j] > np.deg2rad(psdec) - pssa and icdec[j] < np.deg2rad(psdec) + pssa:
            r.append(j)
    evts_p_srad.append([i,r])

    

n_s = Signal Events from point source

In [ ]:
def Pr(s, S, B, N, ns):
    return ns * S / N + (1 - ns/N)*B
#for i in N
# find P[i]

In [ ]:
def L_i(s, S, B, N, ns):
    L = 1
    for i in range(len(N)):
        L *= Pr(s, S[i], B[i],  N[i], ns)
    return L

In [ ]:
def TS(s, S, B, N):
    ts = []
    for i in range(0,4):
        ts.append(2 * np.log10(L_i(s, S, B, N, i)/L(s, S, B, N, 0)))
    return ts

In [ ]:
#Find ns such that TS is maximized